In [ ]:
# 从系统block 一个格点扩展成为两个格点
# 系统块 和环境块 构造超块
# 对超块进行对角化
# 得到约化密度矩阵 
# 得到 transformation_matrix
# 对系统块儿哈密顿量进行重整化

In [1]:
# from one site enlarge to two site, 系统从一个各点扩充到两个各点。
import numpy as np
from scipy.sparse import kron, identity
from collections import namedtuple
def H2(Sz1, Sp1, Sz2, Sp2):  # two-site part of H
    """Given the operators S^z and S^+ on two sites in different Hilbert spaces
    (e.g. two blocks), returns a Kronecker product representing the
    corresponding two-site term in the Hamiltonian that joins the two sites.
    """
    J = Jz = 1.
    return (
        (J / 2) * (kron(Sp1, Sp2.conjugate().transpose()) + kron(Sp1.conjugate().transpose(), Sp2)) +
        Jz * kron(Sz1, Sz2)
    )
if __name__ == "__main__":
    # Model-specific code for the Heisenberg XXZ chain
    model_d = 2  # single-site basis size
    Sz1 = np.array([[0.5, 0], [0, -0.5]], dtype='d')  # single-site S^z
    Sp1 = np.array([[0, 1], [0, 0]], dtype='d')  # single-site S^+
    H1 = np.array([[0, 0], [0, 0]], dtype='d')  # single-site portion of H is zero, 磁场项 或者势能项
    Block = namedtuple("Block", ["length", "basis_size", "operator_dict"])
    EnlargedBlock = namedtuple("EnlargedBlock", ["length", "basis_size", "operator_dict"])
    #初始化一个block
    initial_block = Block(length=1, 
                        basis_size=model_d, 
                        operator_dict={
                        "H": H1,
                        "conn_Sz": Sz1,
                        "conn_Sp": Sp1,})
    block = initial_block
    # enlarges the provided Block by a single site, return an EnlargedBlock
    mblock = block.basis_size
    o = block.operator_dict

    H = kron(o["H"], identity(model_d)) + kron(identity(mblock),H1) + \
        H2(o["conn_Sz"], o["conn_Sp"], Sz1, Sp1) # hamiltonian of block
    conn_Sz = kron(identity(mblock), Sz1) # edge site is enlarged
    conn_Sp = kron(identity(mblock), Sp1) # edge site is enlarged

    enlarged_operator_dict = {"H": H, "conn_Sz": conn_Sz, "conn_Sp": conn_Sp}
    EnlargedBlock(length=(block.length + 1),
                basis_size=(block.basis_size * model_d),
                operator_dict=enlarged_operator_dict) # 存入 EnlareBlock 中。
    print("block hamiltonian H:\n", H)
    print("edge site operator conn_Sz:\n", conn_Sz)
    print("edge site operator conn_Sp:\n", conn_Sp)

block hamiltonian H:
   (0, 0)	0.25
  (1, 1)	-0.25
  (1, 2)	0.5
  (2, 1)	0.5
  (2, 2)	-0.25
  (3, 3)	0.25
edge site operator conn_Sz:
   (0, 0)	0.5
  (0, 1)	0.0
  (1, 0)	0.0
  (1, 1)	-0.5
  (2, 2)	0.5
  (2, 3)	0.0
  (3, 2)	0.0
  (3, 3)	-0.5
edge site operator conn_Sp:
   (0, 1)	1.0
  (2, 3)	1.0


In [19]:
# 构造超块儿哈密顿量
sys_block_2site = EnlargedBlock(length=(block.length + 1), basis_size=(block.basis_size * model_d), 
                                operator_dict=enlarged_operator_dict) # 存入 EnlareBlock 中。
env_block_2site = sys_block_2site # 将系统块变为环境块
# 构造超块哈密顿
m_sys_block = sys_block_2site.basis_size
m_env_block = env_block_2site.basis_size
sys_block_op = sys_block_2site.operator_dict
env_block_op = env_block_2site.operator_dict
#
superblock_Hblock = kron(sys_block_op["H"], identity(m_env_block)) + \
                    kron(identity(m_sys_block), env_block_op["H"]) # 扩大希尔伯特空间
superblock_HConn = H2(sys_block_op["conn_Sz"], sys_block_op["conn_Sp"],
                    env_block_op["conn_Sz"], env_block_op["conn_Sp"]) # 系统块儿和环境块儿两个边缘格点之间的相互作用
superblock_hamiltonian = superblock_Hblock + superblock_HConn 

print("superblock_hamiltonian:", superblock_hamiltonian)

superblock_hamiltonian:   (0, 0)	0.75
  (1, 1)	-0.25
  (1, 2)	0.5
  (1, 4)	0.5
  (2, 1)	0.5
  (2, 2)	0.25
  (3, 3)	0.25
  (3, 6)	0.5
  (4, 1)	0.5
  (4, 4)	-0.25
  (4, 8)	0.5
  (5, 5)	-0.25
  (5, 6)	0.5
  (5, 9)	0.5
  (6, 3)	0.5
  (6, 5)	0.5
  (6, 6)	-0.75
  (6, 10)	0.5
  (7, 7)	0.25
  (7, 11)	0.5
  (8, 4)	0.5
  (8, 8)	0.25
  (9, 5)	0.5
  (9, 9)	-0.75
  (9, 10)	0.5
  (9, 12)	0.5
  (10, 6)	0.5
  (10, 9)	0.5
  (10, 10)	-0.25
  (11, 7)	0.5
  (11, 11)	-0.25
  (11, 14)	0.5
  (12, 9)	0.5
  (12, 12)	0.25
  (13, 13)	0.25
  (13, 14)	0.5
  (14, 11)	0.5
  (14, 13)	0.5
  (14, 14)	-0.25
  (15, 15)	0.75


In [ ]:
# 求得超块儿哈密顿量的基态
# Call ARPACK to find the superblock ground state
from scipy.sparse.linalg import eigsh
(energy,), psi0 = eigsh(superblock_hamiltonian, k=1, which = "SA")
print("energy:\n", energy)
print("psi0:\n", psi0)
print("length of psi0:\n", len(psi0))

In [26]:
# 构造约化密度矩阵
psi0 = psi0.reshape([sys_block_2site.basis_size ,-1], order = "C")
print("psi0:\n", psi0)
rho = np.dot(psi0, psi0.conjugate().transpose())
print("rho:\n", rho)

psi0:
 [[ 1.58269974e-17 -1.60395414e-17  6.90346218e-17  1.49429245e-01]
 [-2.99444996e-17  4.08248290e-01 -5.57677536e-01 -5.29622142e-17]
 [ 1.56625960e-17 -5.57677536e-01  4.08248290e-01 -6.55959717e-17]
 [ 1.49429245e-01 -1.27316366e-16 -6.32092827e-18  2.90022204e-17]]
rho:
 [[ 2.23290994e-02 -5.29612768e-17  2.73262017e-17  6.69879618e-18]
 [-5.29612768e-17  4.77670901e-01 -4.55341801e-01 -5.29262331e-17]
 [ 2.73262017e-17 -4.55341801e-01  4.77670901e-01  7.07614191e-17]
 [ 6.69879618e-18 -5.29262331e-17  7.07614191e-17  2.23290994e-02]]


In [28]:
# 对约化密度矩阵进行对角化处理
# Diagonalize the reduced density matrix and sort the eigenvector by eigenvalue
evals, evecs = np.linalg.eigh(rho)
possible_eigenstates = []
for eval, evec in zip(evals, evecs.transpose()):
    possible_eigenstates.append((eval, evec))
print(possible_eigenstates)
possible_eigenstates.sort(reverse=True, key=lambda x: x[0]) # largest eigenvalue first
print(possible_eigenstates)

[(0.022329099369260197, array([ 0.04790748,  0.38649426,  0.38649426, -0.83603184])), (0.02232909936926026, array([0.04351838, 0.59047373, 0.59047373, 0.54844116])), (0.02232909936926044, array([-0.99790331,  0.04430532,  0.04430532, -0.01621891])), (0.9330127018922189, array([ 3.32583400e-17, -7.07106781e-01,  7.07106781e-01,  9.71445147e-17]))]
[(0.9330127018922189, array([ 3.32583400e-17, -7.07106781e-01,  7.07106781e-01,  9.71445147e-17])), (0.02232909936926044, array([-0.99790331,  0.04430532,  0.04430532, -0.01621891])), (0.02232909936926026, array([0.04351838, 0.59047373, 0.59047373, 0.54844116])), (0.022329099369260197, array([ 0.04790748,  0.38649426,  0.38649426, -0.83603184]))]


In [29]:
# 得到 transformation_matrix
m = 10 # 截断维数
my_m = min(len(possible_eigenstates), m)
transformation_matrix = np.zeros((sys_block_2site.basis_size, my_m), dtype="d", order="F")
for i, (eval, evec) in enumerate(possible_eigenstates[:my_m]):
    transformation_matrix[:,i] = evec
truncation_error = 1 - sum(x[0] for x in possible_eigenstates[:my_m])
print("truncation error:", truncation_error)


truncation error: 2.220446049250313e-16


In [33]:
# 对哈密顿矩阵进行重整化
# Rotate and truncate each operator
print(sys_block_op)
H = sys_block_op["H"]
conn_Sz = sys_block_op["conn_Sz"]
conn_Sp = sys_block_op["conn_Sp"]
#
new_H = transformation_matrix.conjugate().transpose().dot(H.dot(transformation_matrix))
new_conn_Sz = transformation_matrix.conjugate().transpose().dot(conn_Sz.dot(transformation_matrix))
new_conn_Sp = transformation_matrix.conjugate().transpose().dot(conn_Sp.dot(transformation_matrix))
#
new_operator_dict = {}
new_operator_dict["H"] = new_H
new_operator_dict["conn_Sz"] = conn_Sz
new_operator_dict["conn_Sp"] = conn_Sp 
#
newblock = Block(length = sys_block_2site.length,
                basis_size = my_m,
                operator_dict = new_operator_dict
                )

{'H': <4x4 sparse matrix of type '<class 'numpy.float64'>'
	with 6 stored elements in Compressed Sparse Row format>, 'conn_Sz': <4x4 sparse matrix of type '<class 'numpy.float64'>'
	with 8 stored elements (blocksize = 2x2) in Block Sparse Row format>, 'conn_Sp': <4x4 sparse matrix of type '<class 'numpy.float64'>'
	with 2 stored elements in COOrdinate format>}
